In [ ]:
!pip install PyPDF2 huggingface_hub langchain transformers InstructorEmbedding sentence-transformers chromadb openai requests tiktoken faiss-gpu faiss-cpu

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
import requests
import os
from PyPDF2 import PdfReader
import chromadb
import openai

In [ ]:
def download_pdf_from_url(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)


def get_pdf_text(pdf_files):
    text = ""
    for pdf in pdf_files:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


if __name__ == "__main__":
    pdf_urls = [
        'https://assets.grpumps.com/spec_sheets/0700475.pdf',
        'https://assets.grpumps.com/spec_sheets/0350200.pdf'
    ]

    pdf_files = []
    for i, pdf_url in enumerate(pdf_urls):
        filename = f"temp_pdf_{i}.pdf"
        download_pdf_from_url(pdf_url, filename)
        pdf_files.append(filename)

    concatenated_text = get_pdf_text(pdf_files)
    print(concatenated_text)

    # Clean up - delete the temporary PDF files
    for pdf_file in pdf_files:
        os.remove(pdf_file)


In [20]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
source_text = text_splitter.split_text(concatenated_text)

my_api_key = 'Enter OpenAI API Key'

embeddings = OpenAIEmbeddings(openai_api_key = my_api_key)

In [21]:
from langchain.vectorstores import FAISS
db = FAISS.from_texts(source_text, embeddings)

In [25]:
llm = ChatOpenAI(openai_api_key = my_api_key, temperature = 0)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = input("Your question: ")
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)